# NFL 2020 Forecasting Yards Gained

> “The running back takes the handoff… he breaks a tackle…spins… and breaks free! One man to beat! Past the 50-yard-line! To the 40! The 30! He! Could! Go! All! The! Way!”

But will he?

American football is a complex sport. From the 22 players on the field to specific characteristics that ebb and flow throughout the game, it can be challenging to quantify the value of specific plays and actions within a play. Fundamentally, the goal of football is for the offense to run (rush) or throw (pass) the ball to gain yards, moving towards, then across, the opposing team’s side of the field in order to score. And the goal of the defense is to prevent the offensive team from scoring.

In the National Football League (NFL), roughly a third of teams’ offensive yardage comes from run plays. Ball carriers are generally assigned the most credit for these plays, but their teammates (by way of blocking), coach (by way of play call), and the opposing defense also play a critical role. Traditional metrics such as ‘yards per carry’ or ‘total rushing yards’ can be flawed; in this competition, the NFL aims to provide better context into what contributes to a successful run play.

As an “armchair quarterback” watching the game, you may think you can predict the result of a play when a ball carrier takes the handoff - but what does the data say? Deeper insight into rushing plays will help teams, media, and fans better understand the skill of players and the strategies of coaches. It will also assist the NFL and its teams evaluate the ball carrier, his teammates, his coach, and the opposing defense, in order to make adjustments as necessary.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from modeling import *
import pickle
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

## Data Preprocessing

In [2]:
cols = list(pd.read_csv('Data/cleaned_nfl.csv', nrows =1))
nfl = pd.read_csv('Data/cleaned_nfl.csv',
                  usecols =[i for i in cols if i != 'Yard_class'])
nfl

,Team,long_axis,short_axis,speed,accel,Dis,Orientation,Dir,DisplayName,Season,...,Week,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection,TeamAbbr,OppTeamAbbr
0,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,Eric Berry,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
1,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,Allen Bailey,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
2,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,Justin Houston,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
3,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,Derrick Johnson,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
4,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,Ron Parker,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
5,away,75.06,24.00,1.01,0.32,0.18,308.34,95.01,Dee Ford,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
6,away,74.11,16.64,1.11,0.83,0.02,357.23,322.59,Terrance Mitchell,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
7,away,73.37,18.73,1.24,0.74,0.13,328.52,270.04,Phillip Gaines,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
8,away,56.63,26.90,0.26,1.86,0.28,344.70,55.31,Daniel Sorensen,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE
9,away,73.35,38.83,4.55,0.76,0.51,75.47,190.84,Marcus Peters,2017,...,1,Outdoor,Artificial,Clear and warm,63.0,77.0,8.0,sw,KC,NE


In [3]:
nfl['Defenders_vs_Distance'] = nfl['DefendersInTheBox'] / nfl['Distance']

In [6]:
for col in nfl.columns:
    if (nfl[col].dtype == 'O') & (nfl[col].nunique() > 10):
        print(col, nfl[col].nunique())

DisplayName 2230
OffensePersonnel 56
DefensePersonnel 38
Position 18
GameWeather 36
WindDirection 16
TeamAbbr 32
OppTeamAbbr 32


In [4]:
X = nfl.drop('Yards', axis=1)
y = nfl['Yards']

X = pd.get_dummies(X, drop_first=True)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    stratify=y,
                                                    random_state=220)

## Multiple Linear Regression

In [6]:
linreg = LinearRegression().fit(X_train, y_train)
pickle.dump(linreg, open('linreg.sav', 'wb'))

In [34]:
linreg = pickle.load(open('linreg.sav', 'rb'))
print('Model Score:', linreg.score(X_train, y_train))
print('Intercept:', linreg.intercept_)
print('Coefficients:', linreg.coef_)

Model Score: 0.03021867707731174
Intercept: -687.0073150253683
Coefficients: [-0.00038248  0.00019319 -0.16320007 ...  0.0408598  -0.02052368
  0.11086722]


In [10]:
linreg_ols = sm.OLS(y_train,
                    sm.add_constant(X_train)).fit()

In [7]:
model_sum = linreg_ols.summary()
print(model_sum)

                            OLS Regression Results                            
Dep. Variable:                  Yards   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     5.124
Date:                Thu, 21 Nov 2019   Prob (F-statistic):               0.00
Time:                        10:49:36   Log-Likelihood:            -1.3320e+06
No. Observations:              407809   AIC:                         2.669e+06
Df Residuals:                  405343   BIC:                         2.696e+06
Df Model:                        2465                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

```
feats = X_train.columns
vif = [variance_inflation_factor(X_train.values, i)
       for i in range(X_train.shape[1])]
vif_list = zip(feats, vif)
vif_list
```

In [12]:
# Create a dataframe from our regression results
lin_table = linreg_ols.summary().tables[1]
lin_table = pd.DataFrame(lin_table.data)
lin_table.columns = lin_table.iloc[0]
lin_table.drop(0, inplace=True)
lin_table = lin_table.set_index(lin_table.columns[0])
lin_table = lin_table.astype(float)
lin_table

,coef,std err,t,P>|t|,[0.025,0.975]
,,,,,,
const,-686.835000,98.064000,-7.004,0.000,-879.038,-494.632
long_axis,-0.000400,0.000000,-0.962,0.336,-0.001,0.000
short_axis,0.000200,0.001000,0.139,0.889,-0.003,0.003
speed,-0.163200,0.022000,-7.400,0.000,-0.206,-0.120
accel,0.101000,0.012000,8.593,0.000,0.078,0.124
Dis,0.868500,0.202000,4.290,0.000,0.472,1.265
Orientation,0.000050,0.000097,0.512,0.609,-0.000,0.000
Dir,0.000045,0.000096,0.466,0.641,-0.000,0.000
Season,0.374600,0.026000,14.222,0.000,0.323,0.426


In [13]:
lin_table.to_csv('linreg.csv')

In [8]:
lin_table = pd.read_csv('linreg.csv')
lin_table.rename(columns={'Unnamed: 0': 'features'},
                 inplace=True)

In [24]:
lin_table = lin_table.set_index('features')

# Get a list of significant variables
significant_vars = list(lin_table.loc[lin_table['P>|t|']
                                      < .05].index)

# Remove intercept if it's in list
if 'intercept' in significant_vars:
    significant_vars.remove('intercept')

    # Sort by magnitude of coefficients for significant values
lin_table['coef_abs'] = abs(lin_table['coef'])
lin_table = lin_table.loc[lin_table['P>|t|']
                          < 0.05, ].sort_values(by='coef_abs',
                                                ascending=False)

In [27]:
X_train = X_train[list(lin_table.index)[1:]]

linreg_ols_2 = sm.OLS(y_train,
                      sm.add_constant(X_train)).fit()
model2_sum = linreg_ols_2.summary()
print(model2_sum)

                            OLS Regression Results                            
Dep. Variable:                  Yards   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     25.34
Date:                Fri, 22 Nov 2019   Prob (F-statistic):               0.00
Time:                        10:31:25   Log-Likelihood:            -1.3332e+06
No. Observations:              407809   AIC:                         2.667e+06
Df Residuals:                  407410   BIC:                         2.672e+06
Df Model:                         398                                         
Covariance Type:            nonrobust                                         
                                                   coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------

```
model_poly = Pipeline([('scaler', StandardScaler()),
                      ('poly', PolynomialFeatures()),
                      ('linreg', LinearRegression())])

model_poly.fit(X_train, y_train)

print('Model Score:', model_poly.score(X_train, y_train))
print('Intercept:', model_poly.intercept_)
print('Coefficients:', model_poly.coef_)
```

## Lasso

In [15]:
lin_l1 = LassoCV(cv=5, random_state=220)
lin_l1.fit(X_train, y_train)

pickle.dump(lin_l1, open('lasso.sav', 'wb'))

In [16]:
lin_l1 = pickle.load(open('lasso.sav', 'rb'))

print('Model Score:', lin_l1.score(X_train, y_train))
print('Intercept:', lin_l1.intercept_)
print('Coefficients:', lin_l1.coef_)

Model Score: 0.01742781803777027
Intercept: -450.83369364518063
Coefficients: [-0.00019865 -0.         -0.02230502 ... -0.         -0.
  0.        ]


## Ridge

In [17]:
lin_l2 = RidgeCV()
lin_l2.fit(X_train, y_train)

pickle.dump(lin_l2, open('ridge.sav', 'wb'))

In [18]:
lin_l2 = pickle.load(open('ridge.sav', 'rb'))

print('Model Score:', lin_l2.score(X_train, y_train))
print('Intercept:', lin_l2.intercept_)
print('Coefficients:', lin_l2.coef_)

Model Score: 0.029685281616768533
Intercept: -752.0893100047106
Coefficients: [-0.00039245  0.00019759 -0.16117732 ...  0.03487695 -0.01864787
  0.11722674]


## Random Forest Regressor

In [29]:
param_forest = {'criterion': ['mse', 'mae'],
                'n_estimators': [50, 100],
                'max_depth': [6, 20],
                'min_samples_split': [2, 10],
                'min_samples_leaf': [15, 50],
                'max_features': [2, 10, 100]}

gs_forest = run_randomized_search(RandomForestRegressor, param_forest,
                                  X_train, X_test,
                                  y_train, y_test, random_state=220)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

## XGBoost Regressor

In [ ]:
param_xg = {"learning_rate": [0.01, 0.05, 0.1],
            'max_depth': [2, 3, 4],
            'min_child_weight': [15, 16, 17],
            'subsample': [0.2, 0.7, 1],
            'n_estimators': [140, 150, 160]}

gs_boost = run_randomized_search(xgb.XGBRegressor, param_xg,
                                 X_train, X_test,
                                 y_train, y_test, random_state=220)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


## SVM Regressor

In [6]:
scale = StandardScaler()

X_train_scale = scale.fit_transform(X_train)
X_test_scale = scale.transform(X_test)

In [7]:
param_svm = {'C': [0.1, 1, 10],
             'kernel': ['linear', 'rbf', 'poly'],
             'degree': [2, 3, 4]}

gs_svm = run_randomized_search_svr(param_svm, X_train_scale,
                                   X_test_scale, y_train, y_test)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 